In [1]:
import cv2
import pykep
import numpy as np
import imutils
import math
import pandas as pd  # for debug
import csv
from decimal import Decimal

Image tools

In [2]:
def debris_in_pixels(physical_size, vec, AOV=np.radians(39.23), dimensions = (1080, 1920)):
    """
    Returns how many pixels the object at a given distance should take up. Width, height
    :param physical_size: The size of the object
    :param alpha: diagonal angle of view
    :param dimensions: the pixel dimensions of the observer camera used
    :param vec: relative vector from observer to target in body CS.
    """
    pixels_per_length = np.linalg.norm(dimensions)/(2*np.tan(AOV)*float(vec[1]))
    
    return round(physical_size[0]*pixels_per_length), round(physical_size[1]*pixels_per_length)


def resize(original_image, physical_size, vec):
    """
    Resize the image based on the physical size of the target object and the relative distance between 
    target object and observer
    :param physical_size: The size of the object in km
    :param distance: Distance between the target and observer planes
    """
    width, height = debris_in_pixels(physical_size, vec)
    if width == 0 and height ==0:
        return False  # can't resize, object too small
    else:
        return cv2.resize(original_image, (width, height), interpolation=cv2.INTER_AREA)

In [3]:
def to_pixels(r, shape,cols=1920, rows=1080, AOV=np.radians(39.23)):
    """
    Converts physical location to pixels and pixel coordinates. Returns cols,distance,rows
    r is in body CS.
    """
    distance_to_screen = Decimal((rows**2+cols**2)**0.5/(2*np.tan(AOV)))
    unit_r = r/np.linalg.norm(r)
    #print("unit vector", unit_r)
    k = distance_to_screen/unit_r[1]
    r_in_pixels = unit_r*k  # x,y,z --> cols,dist,rows
    #print("location vector in pixels",r_in_pixels)
    
    return round(r_in_pixels[0]+Decimal(cols/2)-Decimal(shape[1]/2)),round(distance_to_screen),round(-r_in_pixels[2]+Decimal(rows/2)-Decimal(shape[0]/2))

def get_left_corner(r, shape):
    """
    Return left corner in rows,cols
    """
    vec = to_pixels(r, shape)
    return (vec[2],vec[0])

In [4]:
def roi_in_frame(left_corner, object_size, frame):
    """
    Return the part of the ROI that is in the frame.
    :left_corner: the coordinates of the left corner of the object in px.
    :object_size: size of the object in pixels in CV2 coordinates (columns, rows)
    """
    frame_size = frame.shape # returns rows,cols
    ROI = [[left_corner[0], left_corner[0] + object_size[0]],[left_corner[1], left_corner[1] + object_size[1]]]
    
    if left_corner[1] + object_size[1] > frame_size[1]:
        #print("outside the right bounding edge")
        # outside the right bounding edge
        ROI[1] = [left_corner[1], frame_size[1]]
    if left_corner[1] < 0:
        #print("outside the left bounding edge")
        # outside the left bounding edge
        ROI[1] = [0, left_corner[1] + object_size[1]]

    if left_corner[0] + object_size[0] > frame_size[0]:
        #print("outside the bottom bounding edge")
        # outside the bottom bounding edge
        ROI[0] = [left_corner[0], frame_size[0]]
    if left_corner[0] < 0:
        #print("outside the upper bounding edge")
        # outside the upper bounding edge
        ROI[0] = [0, left_corner[0] + object_size[0]]

    return frame[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]]


def obj_in_frame(left_corner, frame, img):
    image_size = img.shape
    frame_size = frame.shape
    ROI = [[0,image_size[0]],[0,image_size[1]]]  # the whole image of the object
    if left_corner[1] + image_size[1] > frame_size[1]:
        # outside the right bounding edge
        ROI[1] = [0, frame_size[1] - left_corner[1]]
    if left_corner[1] < 0:
        # outside the left bounding edge
        ROI[1] = [-left_corner[1], image_size[1]]

    if left_corner[0] + image_size[0] > frame_size[0]:
        # outside the bottom bounding edge
        ROI[0] = [0, frame_size[0] - left_corner[0]]
    if left_corner[0] < 0:
        # outside the upper bounding edge
        ROI[0] = [-left_corner[0], image_size[0]]

    return img[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]] 

In [5]:
def create_mask(img):
    return np.zeros(img.shape[:2], dtype="uint8")

Orbital Mechanics

In [6]:
def propagate_based_on_vector(orbit, t,kepler):
    r, v = orbit
    rf, vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    rf = [Decimal(x) for x in rf]
    vf = [Decimal(x) for x in vf]
    return np.array(rf), np.array(vf)

def propagate_based_on_elements(elements, t, kepler):
    r, v = pykep.par2ic(elements, mu=398600)
    if t > 0:
        rf,vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    else:
        rf, vf = r, v
    if kepler == False:  # return vectors
        rf = [Decimal(x) for x in rf]
        vf = [Decimal(x) for x in vf]
        return np.array(rf),np.array(vf)
    if kepler == True:  # return kepler elements
        return pykep.ic2par(rf,vf,mu = 398600)
    
def propagate(orbit,t,kepler=False):
    if len(orbit) == 2:
        return propagate_based_on_vector(orbit,t,kepler)
    if len(orbit) == 6:
        return propagate_based_on_elements(orbit,t, kepler)
    
def get_M(nu,e):
    E_1 = 2 * np.arctan(np.sqrt((1 - e)/(1 + e)) * np.tan(nu / 2))
    M_e1 = E_1 - e * np.sin(E_1)
    return M_e1

Utility

In [7]:
def delete_white_background(img, write=False):
    """
    Takes the images of Debris that I made in paint.
    Inverts the colours and deletes the white background.
    """
    img = np.invert(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, alpha = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    
    # Using cv2.split() to split channels 
    # of coloured image
    b, g, r = cv2.split(img)

    # Making list of Red, Green, Blue
    # Channels and alpha
    rgba = [b,g,r, alpha]

    # Using cv2.merge() to merge rgba
    # into a coloured/multi-channeled image
    dst = cv2.merge(rgba,3)
    print("dst shape", dst.shape)

    # Writing and saving to a new image
    if write is True:
        cv2.imwrite("debris/deb.png", dst)
    return dst        

In [8]:
def add_dict(point1, point2, points):
    dict1 = {"Initial point": point1,
         "Final point": point2}
    points.append(dict1)
    return points

Orbit creation functions


In [9]:
def to_unit_vector(point, width=1080, height=1920 , AOV=np.radians(39.23)):
    """
    y points through the screen and x points right (CCW motion). Point is in cols, rows.
    Output corresponds to rows, cols.
    """
    distance_to_screen = (width**2+height**2)**0.5/(2*np.tan(AOV))
    vector = np.array((point[0]-height/2, distance_to_screen, width/2-point[1]))
    #print("vector in px", vector)
    vector = vector/np.linalg.norm(vector)
    return [Decimal(x) for x in vector]

In [10]:
def get_R(r,v):
    """
    Get the rotation matrix. New y will point in -r direction, new x will point in v direction.
    """
    unit_r = r/np.linalg.norm(r)
    unit_v = v/np.linalg.norm(v)
    
    R = np.array([[Decimal(unit_v[0]), Decimal(-unit_r[0]), Decimal(unit_r[1]*unit_v[2] - unit_v[1]*unit_r[2])],
                [Decimal(unit_v[1]), Decimal(-unit_r[1]), Decimal(unit_v[0]*unit_r[2] - unit_r[0]*unit_v[2])],
                [Decimal(unit_v[2]), Decimal(-unit_r[2]), Decimal(unit_r[0]*unit_v[1] - unit_v[0]*unit_r[1])]])
    R = R.transpose()
    
    return R

def to_body_axis(vec,r, v):
    """
    Calculates the rotation matrix and applies it. Returns the converted vector.
    r and v are used to calculate the rotation matrix and vector is the vector to be 
    rotated.
    """
    R = get_R(r,v)
    
    return R.dot(vec)

def to_global_axis(vec,r,v):
    R = get_R(r,v)
    R = R.transpose()

    return R.dot(vec)

In [11]:
def get_target_r(elementsO, rel_initial, rel_final, point,ti, tof):
    """
    The inputs are the observer elements and target posistion vectors as unit vectors in observer CS.
    For final measurments they are propagated forward in time (t).
    Point corresponds to a chromosome.
    Returns intial and final target posistion vectors.
    """
    # initial "measurment" and orbit
    rho_i = Decimal(point[0])
    rO_initial, vO_initial = propagate(elementsO, ti, kepler=False)
    L_t0 = to_global_axis(rel_initial,rO_initial, vO_initial)
    rT_i = rho_i*L_t0 + rO_initial
    # final "measurment" and orbit
    rho_f = Decimal(point[1])
    rO_final, vO_final = propagate(elementsO, ti+tof, kepler=False)
    L_tf = to_global_axis(rel_final, rO_final, vO_final)
    rT_f = rho_f*L_tf + rO_final
    return rT_i, rT_f

In [12]:
def create_orbits(elements1, orbits, initial_point,final_point,distances,times,TOFs):
    unit_initial = to_unit_vector(initial_point)
    unit_final = to_unit_vector(final_point)
    
    for t in times:
        for distance in distances:
            for TOF in TOFs:
                r_initial, r_final = get_target_r(elements1, unit_initial, unit_final, distance, t, TOF) 
                r_initial = np.array([float(x) for x in r_initial])
                r_final = np.array([float(x) for x in r_final])
                l = pykep.lambert_problem(r1 = r_initial, r2 = r_final, tof = TOF, mu=398600)
                # convert results
                vi = l.get_v1()[0]  # get the velocity vector at initial time for one revolution
                elements = pykep.ic2par(r_initial,vi, mu=398600)  # get Kepler elements from pos and vel vectors
                entry = {}
                entry["Initial time"] = t
                entry["Distance"] = distance
                entry["TOF"] = TOF
                entry["Orbit"] = elements 
                entry["Points"] = [initial_point,final_point]
                orbits.append(entry)
    return orbits

FOV

In [13]:
# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6
 
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ). XYZ
def rotationMatrixToEulerAngles(R) :
 
    assert(isRotationMatrix(R))
 
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
 
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([np.degrees(x), np.degrees(y), np.degrees(z)])

In [14]:
def angle_between(r1,r2):
    """
    Calculate the angle between two vectors
    """
    unit_1 = r1/np.linalg.norm(r1)
    unit_2 = r2/np.linalg.norm(r2)

    return np.arccos(float(np.dot(unit_1,unit_2)))

def behind_earth(r1, r2, R=6371):
    """
    Checks if the target is between the Earth and observer based on 
    a line segment intersecting the sphere.
    Returns true if the target is behind the Earth.
    :param r1: observer
    :param r2: target
    """
    if angle_between(r1,r2) < np.pi/2:  
        return False  # does not intersect
    
    phi = np.arccos(float(np.dot(r1, r2-r1))/(np.linalg.norm(r1)*np.linalg.norm(r2-r1)))
    H = np.linalg.norm(r1)*np.sin(phi)
    
    if H <= R:
        return True  # intersects
    if H > R:
        return False  # does not intersect

def in_FOV(r1, v1, r2, max_vertical=np.radians(21.815), max_horisontal=np.radians(35.436)):
    """
    Checks if the observer should be able to see the target.
    1 for observer. 2 for target.
    Returns True if yes
    """

    rel = r2 - r1
    
    if behind_earth(r1,r2) is True:
        return False
    
    rel_unit = (rel)/np.linalg.norm(rel)
    #print("relative in global", rel)
    
    rel_unit = to_body_axis(rel_unit,r1,v1)
    #print("relative in body", rel_unit)
    if rel_unit[1] < 0:
        return False

    horisontal_angle = abs(np.arctan(float(rel_unit[0]/rel_unit[1])))  
    vertical_angle = abs(np.arctan(float(rel_unit[2]/rel_unit[1])))
    #print("angles (hor,ver)", np.degrees(horisontal_angle),np.degrees(vertical_angle))
    if horisontal_angle < max_horisontal and vertical_angle < max_vertical:
        return True
    else:
        #print("failed cause angles")
        return False

Initialisation

In [89]:
np.degrees(get_M(np.radians(149.512),0.0043))

149.26130630310095

In [16]:
#elements_sat = [7133.8,0.0043,np.radians(98.289), np.radians(49.866), np.radians(356.768), 2.605101240816877]
elements_sat = [7350,0.0037296,np.radians(82.0394),np.radians(41.7294),np.radians(216.168),np.radians(0)]
#elements_sat = [7350,0.0,np.radians(0),np.radians(0),np.radians(0),np.radians(0)]
%store elements_sat

Stored 'elements_sat' (list)


In [242]:
points = []
add_dict([0,1080],[1980,0],points)
add_dict([0,0],[1980,1080],points)
add_dict([790,1080],[1190,0],points)
add_dict([790,0],[1190,1080],points)
add_dict([0,540],[1980,540],points)
add_dict([990,0],[990,1080],points)
add_dict([0,540],[1190,1080],points)
add_dict([0,540],[790,0],points)
points

[{'Initial point': [0, 1080], 'Final point': [1980, 0]},
 {'Initial point': [0, 0], 'Final point': [1980, 1080]},
 {'Initial point': [790, 1080], 'Final point': [1190, 0]},
 {'Initial point': [790, 0], 'Final point': [1190, 1080]},
 {'Initial point': [0, 540], 'Final point': [1980, 540]},
 {'Initial point': [990, 0], 'Final point': [990, 1080]},
 {'Initial point': [0, 540], 'Final point': [1190, 1080]},
 {'Initial point': [0, 540], 'Final point': [790, 0]}]

In [83]:
points = [{'Initial point': [790,1080], 'Final point': [1190,0]}]
distances = [(145,145)]
TOFs = [120]
times = [0]
orbits=[]
for dict1 in points:
    Pi = dict1["Initial point"]
    Pf = dict1["Final point"]
    orbits = create_orbits(elements_sat, orbits,Pi,Pf,distances, times, TOFs)

In [86]:
%store orbits
elements = list(orbits[0]["Orbit"])
for i in range(4):
    i += 2
    elements[i] = np.degrees(elements[i])
print(orbits)
print(elements)

Stored 'orbits' (list)
[{'Initial time': 0, 'Distance': (145, 145), 'TOF': 120, 'Orbit': (7585.077412261296, 0.0522470297561444, 1.3424394437547713, 0.651868556598295, 3.826540733533703, -0.04032983549661335), 'Points': [[790, 1080], [1190, 0]]}]
[7585.077412261296, 0.0522470297561444, 76.91611437903826, 37.34931709036714, 219.24463416637536, -2.3107293624128395]


In [65]:
#to save orbits
with open('Earth/Data_3rd/orbits.csv', 'w') as f:
    w = csv.DictWriter(f, fieldnames=["Initial time", "Distance", "TOF", "Orbit", "Points"])
    w.writeheader()
    w.writerows(orbits)

In [75]:
vs = "Earth/test.avi"
img_path = "debris\deb.png"

In [72]:
object_og = cv2.imread(img_path)
#object_og = np.invert(object_og)  # get white object
#cv2.imshow("object", object_og)
#cv2.waitKey(0)

In [35]:
#Not necessary 
cap = cv2.VideoCapture(vs)  # open the video file
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter("Earth/data_1_{}.avi".format(i), cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))
t = 0
if cap.isOpened() is False:
    print("Error opening video file")


Body

In [87]:
last_points = None
k=-1
excess = 60  # how many seconds after leaving the vid keeps running
last_entry_time =  None
i=0

In [88]:
for dict1 in orbits:
    current_points = dict1["Points"]
    last_entry_time =  None
    if current_points != last_points:
        k+=1
        last_points = current_points
        i=0
    else:
        i+=1
    # choose orbit
    elements_deb = dict1["Orbit"]
    ti = dict1["Initial time"]
    # open the video file
    cap = cv2.VideoCapture(vs)  
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv2.VideoWriter("Earth/data_{}_{}.avi".format(k,i), cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))
    t = 0
    if cap.isOpened() is False:
        print("Error opening video file")

    while cap.isOpened():
        ret0, frame = cap.read()
        print("frame", frame.shape)
        #print(ret0)
        if ret0 is True:
            rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
            rT, vT = propagate_based_on_elements(elements_deb, t-ti, kepler=False)
            t+=1  # move ahead by one second
            #print("Observer pos", rO)
            if in_FOV(rO, vO, rT) is True:
                last_entry_time = t
                rel = rT - rO
                #print("distance", distance)
                rel = to_body_axis(rel, rO, vO)
                #print("relative vec", rel)
                object_resized = resize(object_og, (6,6), rel)  # size of target in pixels
                if object_resized is False:
                    print("video", k,i)
                if object_resized is not False:
                    #print("relative vector in body axis", rel)
                    #print("object size", object_resized.shape)
                    left_corner = get_left_corner(rel,object_resized.shape)  # location of target in the frame
                    print("left corner", left_corner)
                    
                    #print("object resized", object_resized.shape)
                    roi = roi_in_frame(left_corner, object_resized.shape, frame)
                    print("roi shape", roi.shape)
                    # create mask and inverse mask
                    object_in_frame = obj_in_frame(left_corner,frame,object_resized)
                    #cv2.imshow("object in frame", object_in_frame)
                    gray = cv2.cvtColor(object_in_frame, cv2.COLOR_BGR2GRAY)
                    ret, mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
                    mask_inv = cv2.bitwise_not(mask)
                    print("mask shape", mask.shape)
                    #cv2.imshow("mask", mask)
                    #cv2.imshow("inverse mask", mask_inv)
                    # black out the area of object in ROI
                    frame_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                    object_fg = cv2.bitwise_and(object_in_frame, object_in_frame, mask=mask)
                    #print("object_fg", object_fg.shape)
                    #print("frame_bg", frame_bg.shape)
                    #cv2.imshow("frame background", frame_bg)
                    #cv2.imshow("object foreground", object_fg)
                    dst = cv2.add(frame_bg, object_fg)
                    #print("dst", dst.shape)
                    #cv2.imshow("dst", dst)
                    if left_corner[0] <= 0:
                        frame[0:roi.shape[0],left_corner[1]:roi.shape[1]+left_corner[1]] = dst
                    if left_corner[1] <= 0:
                        frame[left_corner[0]:roi.shape[0]+left_corner[0], 0:roi.shape[1]]
                    if left_corner[0] >= 0 and left_corner[1] >= 0:
                        frame[left_corner[0]:roi.shape[0]+left_corner[0],left_corner[1]:roi.shape[1]+left_corner[1]] = dst
                    #cv2.imshow("Earth with ball", frame)
                    cv2.waitKey(0)
            out.write(frame)
        if last_entry_time is not None and t - last_entry_time > excess:
            break

    out.release()
    cap.release()
    cv2.destroyAllWindows()

frame (1080, 1920, 3)
left corner (1050, 760)
roi shape (30, 61, 3)
mask shape (30, 61)
frame (1080, 1920, 3)
left corner (1041, 763)
roi shape (39, 61, 3)
mask shape (39, 61)
frame (1080, 1920, 3)
left corner (1032, 766)
roi shape (48, 61, 3)
mask shape (48, 61)
frame (1080, 1920, 3)
left corner (1023, 769)
roi shape (57, 61, 3)
mask shape (57, 61)
frame (1080, 1920, 3)
left corner (1014, 773)
roi shape (61, 61, 3)
mask shape (61, 61)
frame (1080, 1920, 3)
left corner (1005, 776)
roi shape (61, 61, 3)
mask shape (61, 61)
frame (1080, 1920, 3)
left corner (996, 779)
roi shape (61, 61, 3)
mask shape (61, 61)
frame (1080, 1920, 3)
left corner (987, 783)
roi shape (61, 61, 3)
mask shape (61, 61)
frame (1080, 1920, 3)
left corner (978, 786)
roi shape (61, 61, 3)
mask shape (61, 61)
frame (1080, 1920, 3)
left corner (969, 789)
roi shape (61, 61, 3)
mask shape (61, 61)
frame (1080, 1920, 3)
left corner (960, 793)
roi shape (61, 61, 3)
mask shape (61, 61)
frame (1080, 1920, 3)
left corner (95

Ideal case

In [255]:
all_coordinates = []
last_points = None
k=-1
for dict1 in orbits:
    current_points = dict1["Points"]
    last_entry_time =  None
    if current_points != last_points:
        k+=1
        last_points = current_points
        i=0
    else:
        i+=1
    elements_deb = dict1["Orbit"]
    ti = dict1["Initial time"]
    current_coordinates = {"name": "data_{}_{}".format(k,i),
               "list in body axis":[],
               "list in global axis":[],
               "pixels":[]}
    
    print("Target oribt", elements_deb)
    print(pykep.par2ic(elements_deb, mu=398600))
    print("initial time",ti)

    for t in range(400):
        rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
        rT, vT = propagate_based_on_elements(elements_deb, t-ti, kepler=False)
        if in_FOV(rO, vO, rT) is True:
            rel = rT - rO
            rel2 = rel
            rel = to_body_axis(rel, rO, vO)
            object_resized = resize(object_og, (1,1), rel)  # size of target in pixels
            
            if object_resized is not False:

                centroid = to_pixels(rel,(0,0))
                centroid = [centroid[0], centroid[2], t]
                current_coordinates["pixels"].append(centroid)

                rel_unit = rel/np.linalg.norm(rel)
                rel_unit = np.append(rel_unit,[t])
                current_coordinates["list in body axis"].append(rel_unit)

                rel_unit2 = rel2/np.linalg.norm(rel2)
                rel_unit2 = np.append(rel_unit2,[t])
                current_coordinates["list in global axis"].append(rel_unit2)

    all_coordinates.append(current_coordinates)

Target oribt (10176.599365296699, 0.29649044599350716, 1.3723276955761659, 0.6746877938409508, 3.873427339529676, -0.0791483382724619)
((-4025.5228519107527, -4278.282361123421, -4108.878524340402), (4.838487036356048, 2.1630598185576426, -6.629202878453744))
initial time 0


In [256]:
%store all_coordinates

Stored 'all_coordinates' (list)


Without converting to pixels

In [235]:
all_rel = []
last_points = None
k=-1
for dict1 in orbits:
    current_points = dict1["Points"]
    last_entry_time =  None
    if current_points != last_points:
        k+=1
        last_points = current_points
        i=0
    else:
        i+=1
    elements_deb = dict1["Orbit"]
    ti = dict1["Initial time"]
    current_coordinates = {"name": "data_{}_{}".format(k,i),
               "list in body axis":[],
               "list in global axis":[]}
    print("Target oribt", elements_deb)
    print(pykep.par2ic(elements_deb, mu=398600))
    print("initial time",ti)
    for t in range(400):
        rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
        rT, vT = propagate_based_on_elements(elements_deb, t-ti, kepler=False)
        if in_FOV(rO, vO, rT) is True:
            rel = rT - rO
            rel2 = rel
            rel = to_body_axis(rel, rO, vO)
            object_resized = resize(object_og, (1,1), rel)  # size of target in pixels
            
            if object_resized is not False:
                
                rel_unit = rel/np.linalg.norm(rel)
                rel_unit = np.append(rel_unit,[t])
                current_coordinates["list in body axis"].append(rel_unit)

                rel_unit2 = rel2/np.linalg.norm(rel2)
                rel_unit2 = np.append(rel_unit2,[t])
                current_coordinates["list in global axis"].append(rel_unit2)
                
    all_rel.append(current_coordinates)

Target oribt (10176.599365296699, 0.29649044599350716, 1.3723276955761659, 0.6746877938409508, 3.873427339529676, -0.0791483382724619)
((-4025.5228519107527, -4278.282361123421, -4108.878524340402), (4.838487036356048, 2.1630598185576426, -6.629202878453744))
initial time 0


In [38]:
np.degrees(-0.05892177749825257)

-3.375969172058775

In [57]:
# check intial and final ranges
elements_deb = orbits[0]["Orbit"]
t_initial = all_rel[0]["list in global axis"][0][-1]
t_final = all_rel[0]["list in global axis"][-1][-1]
print("initial time", t_initial)
print("t_final",t_final)
print("tof", t_final - t_initial)
rd,vd = propagate(elements_deb,t_initial,kepler=False)
rs,vs = propagate(elements_sat,t_initial,kepler=False)
rs = np.array(rs)
rd = np.array(rd)
#print("rs", rs)
#print("rd initial", rd)
#print("vd inital", vd)
rel = rd-rs
print("Rho initial",np.linalg.norm(rel))
rd, vd = propagate(elements_deb, t_final,kepler=False)
rs, vs = propagate(elements_sat, t_final,kepler=False)
rs = np.array(rs)
rd = np.array(rd)
#print("rs", rs)
#print("rd final", rd)
rel = rd-rs
print("Rho final",np.linalg.norm(rel))

NameError: name 'all_rel' is not defined

In [238]:
%store all_rel

Stored 'all_rel' (list)


For creating a single list of measurements

In [20]:
Me = get_M(np.radians(66.230),0.0610)
np.degrees(Me)

59.95218508682603

In [17]:
elements_deb = [7290.20,0.0610,np.radians(30.379),np.radians(289.042),np.radians(293.776),1.0463630235301566]
rd,vd = pykep.par2ic(elements_deb, mu=398600)
rs,vs = pykep.par2ic(elements_sat, mu=398600)
rs = np.array(rs)
rd = np.array(rd)
print("rs", rs)
print("rd", rd)
rel = rd-rs
print("Rho initial",np.linalg.norm(rel))
ti = 0
test = []
test2 = []
found = False
for t in range(61):
        rO, vO = propagate(elements_sat, t, kepler=False)
        rT, vT = propagate(elements_deb, t-ti, kepler=False)
        
        rel = rT - rO
        rel2 = rel
        rel = to_body_axis(rel, rO, vO)
        
                
        rel_unit = rel/np.linalg.norm(rel)
        rel_unit = np.append(rel_unit,[t])
        
        test.append(rel_unit)

        rel_unit2 = rel2/np.linalg.norm(rel2)
        rel_unit2 = np.append(rel_unit2,[t])
        test2.append(rel_unit2)
print("Rho final", np.linalg.norm(rT-rO))

rs [-3393.66677619 -4917.17905033  3946.18151386]
rd [ 1980.80461717 -6781.3433417   -199.35311234]
Rho initial 7038.857052135481
Rho final 7113.492574248642686917999944


In [18]:
%store test
%store test2

Stored 'test' (list)
Stored 'test2' (list)


Debug

In [221]:
validating_gen = []
for t in range(80):
    r,v = propagate(elements_sat, t, kepler=False)
    validating_gen.append(r)
%store -r validating_alg

In [227]:
list1 = [[1,2,3],[4,5,6],[7,8,9]]
list2 = [[1,2,3],[4,5,7],[7,8,9]]

for i in range(len(list1)):
    if list1[i] != list2[i]:
        print(f"Lists differ at index {i}: {list1[i]} vs {list2[i]}")


[1, 2, 3]
Lists differ at index 1: [4, 5, 6] vs [4, 5, 7]


In [229]:
a = validating_gen
b = validating_alg

#Compare element-wise and get boolean array
diff = np.equal(a, b)

# Find indices where they differ
indices = np.argwhere(diff == False)

# Print result
print(indices)

[]


In [ ]:
G_CS = []
B_CS = []

corners = []
for t in range(400):
    t+=1
    rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
    rT, vT = propagate_based_on_elements(elements_deb, t, kepler=False)
    print("Observer pos", rO)
    if in_FOV(rO, vO, rT) is True:
        rel = rT - rO
        print("relative vector", rel)
        rel_unit = rel/np.linalg.norm(rel)
        G_CS.append(rel_unit)
        #print("distance", distance)
        rel_unit = to_body_axis(rel_unit, rO, vO)
        rel = to_body_axis(rel, rO, vO)
        B_CS.append(rel_unit)
        object_resized = resize(object_og, (10,10), rel)  # size of target in pixels
        if object_resized is not False:
            print("relative vector in body axis", rel)
            #print("object size", object_resized.shape)
            left_corner = get_left_corner(rel,object_resized.shape)  # location of target in the frame
            print("left corner", left_corner)
            corners.append(left_corner)

In [111]:
G_CS

[array([-0.83927152, -0.47898062, -0.25729531]),
 array([-0.84722385, -0.46824795, -0.25090957]),
 array([-0.8551131 , -0.45723857, -0.24436546]),
 array([-0.86292772, -0.44595104, -0.23766241]),
 array([-0.87065571, -0.4343845 , -0.23080021]),
 array([-0.87828468, -0.42253873, -0.22377901]),
 array([-0.88580184, -0.41041419, -0.21659937]),
 array([-0.8931941 , -0.39801205, -0.20926229]),
 array([-0.90044805, -0.38533428, -0.20176919]),
 array([-0.90755005, -0.37238361, -0.19412199]),
 array([-0.91448629, -0.35916365, -0.18632307]),
 array([-0.92124283, -0.34567888, -0.17837535]),
 array([-0.92780566, -0.33193465, -0.17028225]),
 array([-0.93416081, -0.31793727, -0.16204774]),
 array([-0.9402944 , -0.30369399, -0.15367632]),
 array([-0.94619271, -0.28921298, -0.14517303]),
 array([-0.9518423 , -0.2745034 , -0.13654348]),
 array([-0.95723006, -0.25957534, -0.12779379]),
 array([-0.96234333, -0.24443982, -0.11893062]),
 array([-0.96716995, -0.22910878, -0.10996116]),
 array([-0.97169838,

In [112]:
B_CS

[array([-0.47223817,  0.84308377, -0.25729531]),
 array([-0.46058928,  0.85141171, -0.25090957]),
 array([-0.44864813,  0.85965131, -0.24436546]),
 array([-0.43641362,  0.86779014, -0.23766241]),
 array([-0.42388532,  0.87581533, -0.23080021]),
 array([-0.41106347,  0.88371363, -0.22377901]),
 array([-0.39794905,  0.8914714 , -0.21659937]),
 array([-0.38454383,  0.89907471, -0.20926229]),
 array([-0.3708504 ,  0.90650934, -0.20176919]),
 array([-0.35687221,  0.91376085, -0.19412199]),
 array([-0.34261361,  0.92081466, -0.18632307]),
 array([-0.32807989,  0.92765609, -0.17837535]),
 array([-0.31327731,  0.93427045, -0.17028225]),
 array([-0.2982131 ,  0.94064312, -0.16204774]),
 array([-0.28289548,  0.9467596 , -0.15367632]),
 array([-0.26733368,  0.95260563, -0.14517303]),
 array([-0.25153793,  0.95816729, -0.13654348]),
 array([-0.23551947,  0.96343102, -0.12779379]),
 array([-0.21929049,  0.9683838 , -0.11893062]),
 array([-0.20286414,  0.9730132 , -0.10996116]),
 array([-0.18625448,

In [18]:
%store B_CS

Stored 'B_CS' (list)


In [28]:
corners[15]

(375, 958)

In [29]:
t=0
new_corners = []
for point in corners:
    new_point = [point[1]+112/2, point[0]+112/2]
    new_point.append(t)
    print(new_point)
    new_corners.append(new_point)
    t+=1

[990.0, 540.0, 0]
[992.0, 533.0, 1]
[994.0, 526.0, 2]
[995.0, 519.0, 3]
[997.0, 511.0, 4]
[998.0, 504.0, 5]
[1000.0, 497.0, 6]
[1001.0, 489.0, 7]
[1003.0, 482.0, 8]
[1004.0, 475.0, 9]
[1006.0, 468.0, 10]
[1008.0, 460.0, 11]
[1009.0, 453.0, 12]
[1011.0, 446.0, 13]
[1012.0, 438.0, 14]
[1014.0, 431.0, 15]
[1015.0, 424.0, 16]
[1017.0, 417.0, 17]
[1018.0, 409.0, 18]
[1020.0, 402.0, 19]
[1021.0, 395.0, 20]
[1023.0, 387.0, 21]
[1025.0, 380.0, 22]
[1026.0, 373.0, 23]
[1028.0, 366.0, 24]
[1029.0, 358.0, 25]
[1031.0, 351.0, 26]
[1032.0, 344.0, 27]
[1034.0, 336.0, 28]
[1035.0, 329.0, 29]
[1037.0, 322.0, 30]
[1038.0, 315.0, 31]
[1040.0, 307.0, 32]
[1041.0, 300.0, 33]
[1043.0, 293.0, 34]
[1045.0, 285.0, 35]
[1046.0, 278.0, 36]
[1048.0, 271.0, 37]
[1049.0, 264.0, 38]
[1051.0, 256.0, 39]
[1052.0, 249.0, 40]
[1054.0, 242.0, 41]
[1055.0, 235.0, 42]
[1057.0, 227.0, 43]
[1058.0, 220.0, 44]
[1060.0, 213.0, 45]
[1062.0, 205.0, 46]
[1063.0, 198.0, 47]
[1065.0, 191.0, 48]
[1066.0, 184.0, 49]
[1068.0, 176.0, 

In [30]:
new_corners

[[990.0, 540.0, 0],
 [992.0, 533.0, 1],
 [994.0, 526.0, 2],
 [995.0, 519.0, 3],
 [997.0, 511.0, 4],
 [998.0, 504.0, 5],
 [1000.0, 497.0, 6],
 [1001.0, 489.0, 7],
 [1003.0, 482.0, 8],
 [1004.0, 475.0, 9],
 [1006.0, 468.0, 10],
 [1008.0, 460.0, 11],
 [1009.0, 453.0, 12],
 [1011.0, 446.0, 13],
 [1012.0, 438.0, 14],
 [1014.0, 431.0, 15],
 [1015.0, 424.0, 16],
 [1017.0, 417.0, 17],
 [1018.0, 409.0, 18],
 [1020.0, 402.0, 19],
 [1021.0, 395.0, 20],
 [1023.0, 387.0, 21],
 [1025.0, 380.0, 22],
 [1026.0, 373.0, 23],
 [1028.0, 366.0, 24],
 [1029.0, 358.0, 25],
 [1031.0, 351.0, 26],
 [1032.0, 344.0, 27],
 [1034.0, 336.0, 28],
 [1035.0, 329.0, 29],
 [1037.0, 322.0, 30],
 [1038.0, 315.0, 31],
 [1040.0, 307.0, 32],
 [1041.0, 300.0, 33],
 [1043.0, 293.0, 34],
 [1045.0, 285.0, 35],
 [1046.0, 278.0, 36],
 [1048.0, 271.0, 37],
 [1049.0, 264.0, 38],
 [1051.0, 256.0, 39],
 [1052.0, 249.0, 40],
 [1054.0, 242.0, 41],
 [1055.0, 235.0, 42],
 [1057.0, 227.0, 43],
 [1058.0, 220.0, 44],
 [1060.0, 213.0, 45],
 [10

In [31]:
%store new_corners

Stored 'new_corners' (list)
